In [ ]:
#--- imports and dependencies
import mne, os
import numpy as np
import pandas as pd
import matplotlib
from helpers import *     # note: this script imports functions from helpers.py
from os.path import join

%gui qt
mne.set_log_level(verbose='WARNING')
matplotlib.use('Qt5Agg')

print(ROOT) # defined in helpers

In [ ]:
#--- set paths within workspace, ideally the ROOT folder would be open in your code editor
pilot = []
excluded = [] + pilot   # see exclusion criteria
#--- list of subjects, can also be added manually
subjects = list({i[:5] for i in os.listdir(raw_dir) if i.startswith("R") and i[:5] not in excluded}) # raw_dir defined in helpers
print('N =', len(subjects))
print(subjects)

In [ ]:
#--- work with a single subject until epoching
#--- find events (sanity check)
subj = input("Participant ID: ").strip()
print(subj)
raw = get_meg_data(subj, "raw")
events = mne.find_events(raw, min_duration=0.002)
print('events: ', len(events))
print(events)

In [ ]:
#--- filter data (1-40 Hz bandpass)
print('filtering...')
raw.filter(1,40, method='iir')

In [ ]:
#--- plot after filtering (IN THESE GRAPHS: annotate bad time ranges, radio bursts, body movements, and bad channels)
raw.plot_psd()
raw.plot()

In [ ]:
#--- interpolate bad channels and add annotations
print('bads: %s'%raw.info['bads'])
print('interpolating bads...')
raw.interpolate_bads()

In [ ]:
#--- create, fit, plot ica model to data (explains 95% of variance over n_components)
ica = mne.preprocessing.ICA(n_components=0.95, method='fastica', random_state = 3334) #--- note: random state here allows you to replicate an ICA model, is arbitrary number
print('fitting ica...')
ica.fit(raw, reject_by_annotation=True) # this rejects your annotations from raw.plot() above

#--- plot ica components in gui for deselection
ica.plot_sources(raw) # timecourses of components --- still search in here for radio bursts
ica.plot_components() # topographies of components --- look for eye blinks, saccades, heartbeat, iron cross (see wiki for examples)

In [ ]:
#--- confirm that ica components you selected are excluded
ica.exclude

In [ ]:
#--- apply the model to the data, excluding bad components
raw = ica.apply(raw, exclude=ica.exclude) # this replaces the raw object! be wary!
print('saving...')
save_meg_data(subj, raw, "ica-raw")

del raw, ica #--- this will remove the variables, be wary!

In [ ]:
#--- epoch constants
epoch_tmin = -0.1               # 100 ms of pre-stimulus blank screen
epoch_tmax = 0.8                # 300 ms of on-screen stimulus, 500ms of post-stimulus blank screen
epoch_baseline = (-0.1, 0)      # where to use baseline in mne.Epochs()

dt = 0.001                      # timestep (1ms)
standard_onset = 60             # consensus for KIT-NYU MEG photodiode onset delay from trigger (60ms)
decim = 1                       # decim > 1 will downsample your data by that factor. decim = 1 does not downsample
reject = {'mag': 3e-12}         # reject sensor activity above threshold (radio bursts, not neural data)

In [ ]:
#--- load in ICA raw data, logfile, and shift events to photodiode onset
#--- create and reject epochs by high amplitude threshold
print(subj)
print('loading data...')
raw = get_meg_data(subj, 'ica-raw') # processed data
log = get_log_file(subj)

print('saving photodiode...')
shifted_events = apply_photodiode(subj, raw)
print(len(shifted_events))

picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=False, stim=False)
print('creating epochs...')
epochs = mne.Epochs(raw, shifted_events, event_id=inflection_map, # condition-event mapping dictionary in helpers
                tmin=epoch_tmin, tmax=epoch_tmax,
                baseline=epoch_baseline, picks=picks_meg,
                metadata=log, decim=decim, preload=True)

print('noise threshold rejection...')
_len_epochs = len(epochs)       # temp variable to calculate rejection count
epochs.drop_bad(reject)
n_threshold_rejection = _len_epochs - len(epochs)
print(n_threshold_rejection, 'dropped')

#--- apply behavioral filtering
epochs = epochs[(epochs.metadata['RT'] > 0.2) & (epochs.metadata['RT'] < 2)] # attentiveness thresholds
print(len(epochs))

#--- check evoked data, then save epochs
get_evokeds(subj, epochs)
save_meg_data(subj, epochs, 'ica-epo')

In [ ]:
for i, subj in enumerate(subjects):
    print("Computing STCs for subject (%s/%s)" % (str(i + 1), len(subjects)))

    #--- files from coregistration
    trans_fname = os.path.join(meg_dir, '%s-trans.fif' % subj)
    trans = mne.read_trans(trans_fname)
    src_fname = os.path.join(subjects_dir, subj, 'bem', '%s-ico-4-src.fif' % subj)
    src = get_source_space(subj, src_fname, force_new=False)
    bem_fname = os.path.join(subjects_dir, subj, 'bem', '%s-inner_skull-bem-sol.fif' % subj)

    #--- get meg data 
    epoch_file = os.path.join(meg_dir, subj, f'{subj}_{expt}-ica-epo.fif')
    epochs = mne.read_epochs(epoch_file)
    print(len(epochs))

    info = epochs.info
    
    #--- inverse model calculation
    evokeds = get_evokeds(subj, epochs) # this should plot and save the ev to check
    bem = get_BEM(subj, bem_fname)
    fwd = get_forward_solution(subj, info, src, trans, bem_fname, force_new=False)
    cov = get_covariance_matrix(subj, epochs, force_new=False)
    inv, lambda2 = get_inverse_operator(info, fwd, cov)

    create_and_save_STCs(subj, evokeds, inv, lambda2)

    #--- delete variables
    del info, trans, src, fwd, cov, inv, evokeds